<a href="https://colab.research.google.com/github/ParitoshP702/Bilevel-Optimization/blob/main/Genetic_Algorithm(MNIST).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import random
import tensorflow as tf
import scipy as spy

from operator import itemgetter

from tqdm import tqdm

from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import regularizers

from keras import backend as K

In [ ]:
pip install gurobipy

In [ ]:
import gurobipy as gp

In [ ]:
 params = {
  "WLSACCESSID": '753e7886-7142-449d-8baa-d41ca78716ef',
  "WLSSECRET": '880d2525-364b-41d0-ac23-6dcf7ad15312',
  "LICENSEID": 914249,
  }
env = gp.Env(params=params)

In [ ]:
# from google.colab import drive
# drive.mount("/content/drive")
# #Use ! for global command.
# !pwd
# #Use % for the current shell. Note that cd will not work with !.
# %cd "/content/drive/MyDrive/csv files/IIM_A/loanacceptance.csv"

In [ ]:
def complete_weight_array(model):
  weights_list = []
  for i in range(len(model.weights)):
    weights_array = tf.make_ndarray(tf.make_tensor_proto(model.weights[i]))
    if i%2 == 0:
      shape_array = weights_array.shape
      for j in range(shape_array[0]):
        for k in range(shape_array[1]):
          weights_list.append(weights_array[j][k])
          # if len(weights_list_new) < skip_length:
          #   weights_list_new.append(0)
          # else:
          #   weights_list_new.append(weights_array[j][k])
    else:
      lgt = weights_array.shape[0]
      for j in range(lgt):
        weights_list.append(weights_array[j])
        # if len(weights_list_new) < skip_length:
        #   weights_list_new.append(0)
        # else:
        #   weights_list_new.append(weights_array[j])
  return np.array(weights_list)

In [ ]:
def weight_array_for_hessian(model):
  skip_length = len(model.layers[0].weights[0].numpy().reshape(-1)) + len(model.layers[0].weights[1].numpy().reshape(-1))
  weights_list = []
  for i in range(len(model.weights)):
    weights_array = tf.make_ndarray(tf.make_tensor_proto(model.weights[i]))
    if i%2 == 0:
      shape_array = weights_array.shape
      for j in range(shape_array[0]):
        for k in range(shape_array[1]):
          # weights_list.append(weights_array[j][k])
          if len(weights_list) < skip_length:
            weights_list.append(0)
          else:
            weights_list.append(weights_array[j][k])
    else:
      lgt = weights_array.shape[0]
      for j in range(lgt):
        # weights_list.append(weights_array[j])
        if len(weights_list) < skip_length:
          weights_list.append(0)
        else:
          weights_list.append(weights_array[j])
  return np.array(weights_list)

In [ ]:
def compute_gradient(x_target,y_target,model):##general function which returns the list of gradient vector as an numpy array
  with tf.GradientTape() as tape:
    loss_object = tf.keras.losses.MeanSquaredError()
    y_pred_array = model(x_target,training = True)
    loss = loss_object(y_target,y_pred_array)
  g = tape.gradient(loss,model.trainable_variables)
  final_grad_list = []
  for i in range(len(g)):
    grad_array = tf.make_ndarray(tf.make_tensor_proto(g[i]))
    if i%2==0:
      grad_shape = grad_array.shape
      for j in range(grad_shape[0]):
        for k in range(grad_shape[1]):
          final_grad_list.append(grad_array[j][k])
    else:
      length = grad_array.shape[0]
      for j in range(length):
        final_grad_list.append(grad_array[j])
  return np.array(final_grad_list)


In [ ]:
def compute_hessian(model):
  final_hessian_list = []
  with tf.GradientTape(persistent = True) as tape1:
    with tf.GradientTape(persistent = True) as tape2:
      loss_object = tf.keras.losses.CategoricalCrossentropy()
      y_pred_array = model(x_train,training = True)
      loss = loss_object(y_training,y_pred_array)
    g = tape2.gradient(loss, model.trainable_variables)
  for i in range(len(g)):
    # reshaped_grad = tf.reshape(g[i], [-1])
    h = tape1.jacobian(g[i],model.trainable_variables)
    final_hessian_list.append(h)


  ##Now this final hessian list is actually a double dimensional list of tensors, so we will convert it into a matrix
  #reshaping the double dimensional list of tensors into a matrix
  hessian_matrix = np.empty(shape = (1,1),dtype = float)
  for i in range(len(final_hessian_list)):
    hess_col_mat = np.empty(shape = (1,1),dtype = float)
    for j in range(len(final_hessian_list[i])):
      hess_array = tf.make_ndarray(tf.make_tensor_proto(final_hessian_list[i][j]))
      hess_shape = hess_array.shape
      if i%2 == 0:
        if j%2 == 0:
          hess_array = hess_array.reshape(hess_shape[0]*hess_shape[1],hess_shape[2]*hess_shape[3])
        else:
          hess_array = hess_array.reshape(hess_shape[0]*hess_shape[1],hess_shape[2])
      else:
        if j%2 == 0:
          hess_array = hess_array.reshape(hess_shape[0],hess_shape[1]*hess_shape[2])
        else:
          hess_array = hess_array
      if j==0 :
        hess_col_mat = hess_array
      else:
        hess_col_mat = np.concatenate((hess_col_mat,hess_array),axis = 1)
    if i==0:
      hessian_matrix = hess_col_mat
    else:
      hessian_matrix= np.concatenate((hessian_matrix,hess_col_mat),axis = 0)


  return hessian_matrix



In [ ]:
def perform_fine_tuning(model,params_model):
  number_of_layers = params_model[1]
  reg_param = params_model[3]
  neurons_per_layer = params_model[0]
  activation_fun = params_model[2]

  ###calculating the hessian for the model and the gradient of the validation loss
  hessian_matrix = compute_hessian(model)
  grad_validation = compute_gradient(x_eval,y_val_array,model)
  final_weights_array_new = weight_array_for_hessian(model)
  l = len(final_weights_array_new)


  ##adding the regularization term in the hessian
  weight_array_vec = final_weights_array_new.reshape(l,1)/len(y_train)
  hessian_col_mat = np.concatenate((weight_array_vec,hessian_matrix),axis = 1)
  weight_array_withreg = np.concatenate(([[0]],final_weights_array_new.reshape(1,l)),axis = 1)/len(y_train)
  hessian_mat_with_reg = np.concatenate((weight_array_withreg,hessian_col_mat),axis = 0)


  grad_validation_new = np.concatenate(([[0]],grad_validation.reshape(1,l)),axis = 1)#validation array with regularization


  ##Solving the linear program
  ub = [10 for i in range(l+1)]
  lb = []
  for i in range(l+1):
    if i==0:
      lb.append(1e-5)
    else:
      lb.append(-10)


  # Create the model within the Gurobi environment
  m = gp.Model(env=env)
  # m = gp.Model()
  x = m.addMVar((l+1,),lb = lb, ub = ub )
  m.setObjective(grad_validation_new@x)
  # m.addConstr(hessian_mat_with_reg@x == 0)
  m.addConstr(hessian_mat_with_reg@x <= 0.1)
  m.addConstr(hessian_mat_with_reg@x >= -0.1)
  x.PStart = np.zeros(l+1)
  # GRBModel.Set(Pstart = np.zeros(l+1))
  m.optimize()
  all_vars = m.getVars()
  values = m.getAttr("x",all_vars)
  values = np.array(values)
  values = values/np.linalg.norm(values)

  final_weights_array = complete_weight_array(model)
  weight_array_with_reg = np.concatenate(([[reg_param]],final_weights_array.reshape(1,l)),axis = 1).reshape(-1)
  descent_factors = []
  for i in range(-100,20000,200):
    descent_factors.append(i*1e-3)
  descent_factors = np.array(descent_factors)


  weight_sample_space_matrix = np.empty(shape = (len(descent_factors),len(weight_array_with_reg)),dtype = float)##initializing the weight sample space matrix
  for i in range(len(descent_factors)):
    weight_sample_space_matrix[i] =weight_array_with_reg+ values*descent_factors[i]   ##assigning values to the weight sample space matrix


  ##defining the loss object
  loss_object = tf.keras.losses.CategoricalCrossentropy()

  ##computation for validation loss
  def validation_loss_computation(weight_and_reg_array):##function which computes the loss score of the model corresponding to given weights

      model_demo = Sequential()
      model_demo.add(Dense(units = 3, input_dim = 784))
      for i in range(number_of_layers):
          model_demo.add(Dense(units = neurons_per_layer, activation = "relu", kernel_regularizer = tf.keras.regularizers.L2(weight_and_reg_array[0])))
      model_demo.add(Dense(units = 10,activation = "softmax", kernel_regularizer = tf.keras.regularizers.L2(weight_and_reg_array[0])))
      model_demo.compile(loss = tf.keras.losses.CategoricalCrossentropy(), optimizer = "Adam", metrics = ["accuracy"])
      weight_tracker = 1##as "weight_and_reg_array" is a one dimensional array it keeps track of the indices of the array
      for i in range(len(model_demo.layers)):##changing the weights of the model layer wise
        orignal_weight_list = model.layers[i].weights
        array_1 = orignal_weight_list[0].numpy()##array corresponding to the weight matrix
        array_2 = orignal_weight_list[1].numpy()##array corresponding to the bias vector
        array_1_new = weight_and_reg_array[weight_tracker:weight_tracker+array_1.shape[0]*array_1.shape[1]]
        weight_tracker += array_1.shape[0]*array_1.shape[1]##updating the weight tracker
        array_2_new = weight_and_reg_array[weight_tracker:weight_tracker + array_2.shape[0]]
        weight_tracker += array_2.shape[0] #updating the weight tracker
        array_1_new = array_1_new.reshape(array_1.shape) ##new weight matrix
        array_2_new = array_2_new.reshape(array_2.shape) ##new bias vector
        list_of_new_array = [] ##list of the new weight matrix and the new bias vector
        list_of_new_array.append(array_1_new)
        list_of_new_array.append(array_2_new)
        model_demo.layers[i].set_weights(list_of_new_array) ##appending the new weights into the given layer of the model
      y_pred_array = model_demo(np.array(x_eval),training = False)
      y_pred_training = model_demo(np.array(x_train),training = False)
      loss = loss_object(y_val_array,y_pred_array)
      loss_t = loss_object(y_training,y_pred_training)
      # loss1,_ = model_demo.evaluate(x_eval,y_eval,verbose= 0)
      # loss2,_= model_demo.evaluate(x_train,y_train,verbose = 0)
      return loss,loss_t,model_demo


  loss_array_valid = np.empty(shape = len(descent_factors),dtype = float)##array to contain the training losses
  loss_array_train = np.empty(shape = len(descent_factors),dtype = float)##array to contain the validation losses

  for i in range(len(loss_array_valid)):
    loss_array_valid[i] ,loss_array_train[i],_= validation_loss_computation(weight_sample_space_matrix[i])


  ideal_weight_array = weight_sample_space_matrix[loss_array_valid.argmin()]
  ideal_regularization_parameter = ideal_weight_array[0]
  _,_,best_model = validation_loss_computation(ideal_weight_array)

  return ideal_weight_array[0],best_model,loss_array_valid.min()



In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# weight_array_for_hessian(models[1])

Loading and Preprocessing the dataset

In [ ]:
(X_train,Y_train) , (X_test,Y_test) = tf.keras.datasets.mnist.load_data(path = "mnist.npz")

In [ ]:
X_train = X_train/255.0
X_test = X_test/255.0

In [ ]:
train_count = 5000
eval_count = 2500

In [ ]:
X_test = X_test[:eval_count,:,:]

In [ ]:
x_test = X_test.reshape(X_test.shape[0],-1)

In [ ]:
y_test = Y_test[:eval_count]

In [ ]:
x_train = X_train[:train_count,:,:]
x_eval = X_train[train_count:train_count+eval_count,:,:]
y_train = Y_train[:train_count]
y_eval = Y_train[train_count:train_count+eval_count]

In [ ]:
x_train = x_train.reshape(x_train.shape[0],-1)
x_eval = x_eval.reshape(x_eval.shape[0],-1)

In [ ]:
y_training = np.zeros(shape = (len(y_train),10))

In [ ]:
for i in range(len(y_train)):
  # y_training[i][y_train_array[i]] = 1
  for j in range(10):
    if j != (y_train[i]):
      y_training[i][j] = 0.0
    else:
      y_training[i][j] = 1.0

In [ ]:
y_val_array = np.empty(shape = (len(y_eval),10),dtype = float)

In [ ]:
for i in range(len(y_eval)):
   for j in range(10):
    if j != (y_eval[i]):
      y_val_array[i][j] = 0.0
    else:
      y_val_array[i][j] = 1.0

In [ ]:
#

In [ ]:
def hyperparameters(returnAs='vals'):
    parameters = {}

    #Add other parameters here

    parameters["neurons_per_layer"] = [10,20,30,40,50]
    parameters["number_of_layers"] = [3,4,5]
    parameters["activation_function"] = ['relu', 'tanh', 'sigmoid']
    parameters["regularization_parameter"] = [1e-10,1e-9,1e-8,1e-7,1e-6]
    #Search over regularization parameter as well
    #parameters["regularization"] = []

    #Keep the last one as optimizer
    #parameters["optimization_method"] = ['adam', 'rmsprop']
    # parameters["optimization_method"] = ['adam']

    if returnAs == 'dict': return(parameters)
    if returnAs == 'vals': return(list(parameters.values()))
    if returnAs == 'keys': return(list(parameters.keys()))

# def hyperparameters_old():
#     parameters = []
#     units_per_layer = [5, 10, 15]
#     layers = [1, 2, 3]
#     activation = ['relu', 'tanh', 'sigmoid']
#     optimizer = ['adam', 'rmsprop']
#     parameters.append(units_per_layer)
#     parameters.append(layers)
#     parameters.append(activation)
#     parameters.append(optimizer)

#     return(parameters)

In [ ]:
hyperparameters()

In [ ]:
hyperparameters('dict')

In [ ]:
def generate_population(size):
    parameters = hyperparameters()

    population = []
    i=0
    while i < size:
        individual = [random.choice(parameters[j]) for j in range(len(parameters))]
        if individual not in population:
            population.append(individual)
            i+=1
    return(population)

In [ ]:
generate_population(10)

In [ ]:
def new_child(parent1, parent2):
    parent_size = len(parent1)
    rint = random.randint(0, parent_size)
    #child = [random.choice([parent1[i],parent2[i]]) for i in range(parent_size)]
    child1 = parent1[:rint]+parent2[rint:]
    child2 = parent2[:rint]+parent1[rint:]
    child = random.choice([child1,child2])

    return(child)

In [ ]:
parent1 = [15, 1, 'relu', 1e-10]
parent2 = [10, 3, 'sigmoid', 1e-9]
new_child(parent1,parent2)

In [ ]:
def mutation(population):
    parameters = hyperparameters()
    for chromosome in population:
        if random.random() < 0.1 :
            key = random.choice(range(len(parameters)))
            parameters = hyperparameters()
            mutate_key = random.choice(parameters[key])
            chromosome[key] = mutate_key

    return(population)

In [ ]:
loss_object = tf.keras.losses.CategoricalCrossentropy()

In [ ]:
def train_model(parameters,initialWeights=None):
    neurons_per_layer = parameters[0]
    no_of_layers = parameters[1]
    # activation_function = parameters[2]

    #Following is not used here
    # optimization_method = parameters[3]
    regularization_param = parameters[3]

    model = Sequential()
    model.add(Dense(units=3, input_dim=784))

    for _ in range(no_of_layers):
        model.add(Dense(units=neurons_per_layer, activation="relu",kernel_regularizer = tf.keras.regularizers.L2(regularization_param)))

    model.add(Dense(units = 10,  activation = 'softmax',kernel_regularizer = tf.keras.regularizers.L2(regularization_param)))




    return(model)

def evaluate_model(individual,initialWeights=None):
    model = train_model(individual,initialWeights)

    #The last element in the individual should always be the optimizer
    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer="Adam", metrics=['accuracy'])
    model.fit(x_train, y_training, batch_size = 64, epochs = 10)

    # print("Training Accuracy:", model.evaluate(x_train, y_train, verbose = 0)[1])

    #Evaluate on evaluation data
    # loss_score, accuracy_score = model.evaluate(x_eval, y_val_array, verbose = 0)
    y_pred = model(x_eval,training = False)
    loss_score = loss_object(y_pred,y_val_array)
    return loss_score,model

In [ ]:
generations = 15
population_size = 10

initial_population = generate_population(population_size)
losses = []
models = []
# losses = [evaluate_model(individual) for individual in initial_population]
for individual in initial_population:
  loss,model = evaluate_model(individual)
  losses.append(loss)
  models.append(model)
for i in range(len(models)):
  initial_population[i][3],models[i],losses[i] = perform_fine_tuning(models[i],initial_population[i])

In [ ]:
initial_population

In [ ]:
initial_population[0][3]

In [ ]:
print(initial_population)
print(losses)

In [ ]:
generation_wise_best_loss = []

In [ ]:
#Append fitness to population
population_with_fitness = [pop+[f] for pop,f in zip(initial_population,losses,models)]

#Write the generation steps here
for _ in tqdm(range(generations)):
    #Includes start as well as end while generating random integers
    r1 = random.randint(0, population_size-1)
    r2 = random.randint(0, population_size-1)

    parent1 = population_with_fitness[r1][0:-1]
    parent2 = population_with_fitness[r2][0:-1]

    number_of_offspring = 2
    offspring = [new_child(parent1,parent2) for i in range(number_of_offspring)]
    offspring = mutation(offspring)

    offspring_losses = []
    offspring_models = []

    for individual in offspring:
      loss,model = evaluate_model(individual)
      offspring_losses.append(loss)
      offspring_models.append(model)
    for i in range(len(offspring_losses)):
      offspring[i][3],offspring_models[i],offspring_losses[i] = perform_fine_tuning(offspring_models[i],offspring[i])
    # offspring_fitness = [evaluate_model(individual) for individual in offspring]
    # models.extend(offspring_models)
    offspring_with_fitness = [pop+[f] for pop,f in zip(offspring,offspring_losses,offspring_models)]

    population_with_fitness.extend(offspring_with_fitness)

    #Sort in descending by fitness
    population_with_fitness.sort(key = lambda i: i[-1])

    #Keep the best members
    population_with_fitness = population_with_fitness[0:population_size]
    generation_wise_best_loss.append(offspring_with_fitness.sort(key = lambda i:i[-1])[0][4])

best_individual = population_with_fitness[0][0:-1]
best_fitness = population_with_fitness[0][-1]

In [ ]:
best_individual


In [ ]:
best_fitness

In [ ]:
generation_wise_best_loss

In [ ]:
import pickle

In [ ]:
file_to_store = open("GA_MNIST(with_lp).txt","wb")
pickle.dump(generation_wise_best_loss, file_to_store)
file_to_store.close()

In [ ]:
best_model = best_individual[5]

In [ ]:
file_to_store = open("GA_MNIST_best_model.txt","wb")
pickle.dump(best_model,file_to_store)
file_to_store.close()

In [ ]:
s = evaluate_model(best_individual)
s

In [ ]:
population_with_fitness

In [ ]:
best_individual

In [ ]:
neurons_per_layer = best_individual[0]
no_of_layers = best_individual[1]
activation_function = "relu"
# optimization_method = best_individual[3]
regularization_param = best_individual[3]

In [ ]:
model = Sequential()
model.add(Dense(units=6, input_dim=784))

for _ in range(no_of_layers):
  model.add(Dense(units=neurons_per_layer, activation=activation_function, kernel_regularizer = tf.keras.regularizers.L2(regularization_param)))
model.add(Dense(units = 10,  activation = 'softmax', kernel_regularizer = tf.keras.regularizers.L2(regularization_param)))

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer="Adam", metrics=['accuracy'])
model.fit(x_train, y_training, batch_size = 64, epochs = 10)

# loss_score, accuracy_score = model.evaluate(x_eval,y_eval, verbose = 0)
y_pred = model(x_eval,training = False)
loss_score = loss_object(y_pred,y_val_array)

In [ ]:
# accuracy_score##accuracy score for the best fit model

In [ ]:
loss_score# model.trainable_parameters

In [ ]:
y_pred_testing = model(x_test,training = False)

In [ ]:
y_test_array = np.zeros(shape = (len(y_test),10),dtype = float)##trying to one hot encode the array for finding the cross entropy loss
for i in range(len(y_test)):
  for j in range(10):
    if j != (y_test[i]):
      y_test_array[i][j] = 0.0
    else:
      y_test_array[i][j] = 1.0


In [ ]:
loss_object(y_test_array,y_pred_testing)##we are finding the loss this way because we calculated the gradient and hessian from this kind of loss (and our entire analysis depends on this)